# Hallucination Detection

**The method adopts the concept of RefChecker and is primarily divided into Extractor and Checker components. However, I am unable to directly apply the GitHub method (https://github.com/amazon-science/RefChecker/tree/main/refchecker) due to the following issues:**

1. The GitHub implementation relies on the OpenAI API, which requires a proxy for usage in China.
2. RefChecker does not achieve the goal of hallucination position detection.
3. RefChecker uses its own set of references for comparison, but the references I have are limited, some are inaccessible and just can be the context.

**Based on the above, I only use the conceptual framework of RefChecker and some of its key prompts for improvement and optimization. In my method, I primarily use the Anthropic API (Claude).**

**My idea of solving the problems:**
1. **Claim Extraction:** Claims are extracted by using the Anthropic API to process both `model_output_text` and `model_input`. Each claim is a merged representation derived from a triple-structured knowledge breakdown. (This uses prompts from RefChecker's Extractor component).

2. **Primary Reference Acquisition:** Based on the claims obtained in Step 1, I use a self-verification approach. Each claim is processed through the Anthropic API for self-verification to obtain factually consistent statements and even supplemental content. The returned statements are combined with external knowledge I retrieved earlier using the Google API. This forms a complete set of references for subsequent verification checks.

3. **Verification of Claims, Model Input, Model Output Text, and References:** The extracted claims, references from Step 2, and `model_input` are verified. The Anthropic API is employed with specific prompts to identify hallucinated words in `model_output_text` and estimate their hallucination probability.

4. **Soft Label Generation:** Based on the hallucinated words identified above, their positions in `model_output_text` are mapped. Using the probabilities obtained, a set of `soft_labels` is generated.

In [ ]:
import json
import os
import anthropic
from tqdm import tqdm
import requests
import httpx
from RefChecker.refchecker.extractor import extractor_prompts
from scorer import recompute_hard_labels
import glob

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
os.environ['ANTHROPIC_API_KEY'] = 'sk-an' # set the Anthropic key

# setting proxies (cause in China)
proxies = {
    "http": "http://127.0.0.1:10809",
    "https": "http://127.0.0.1:10809",
    "socks5": "socks5://127.0.0.1:10808"  # SOCKS5 proxy
}

session = requests.Session()
session.proxies.update(proxies)

# setting Anthropic API key
api_key = os.getenv("ANTHROPIC_API_KEY")

## 1. Extracting Claims (Extractor) - Each claim is a merger of triple-structured knowledge.

In [ ]:
LLM_TRIPLET_EXTRACTION_PROMPT_Q = extractor_prompts.LLM_TRIPLET_EXTRACTION_PROMPT_Q
LLM_Triplet_To_Claim_PROMPT_Q = extractor_prompts.LLM_Triplet_To_Claim_PROMPT_Q
LLM_CLAIM_EXTRACTION_PROMPT_Q = extractor_prompts.LLM_CLAIM_EXTRACTION_PROMPT_Q

In [ ]:
url_anthropic = "https://api.anthropic.com/v1/messages"
def extract_triplets_to_claims(question, model_output_text):
    prompt = LLM_CLAIM_EXTRACTION_PROMPT_Q.format(q=question, r=model_output_text)
    headers = {
        "x-api-key": api_key,
        "content-type": "application/json",
        "anthropic-version": "2023-06-01"
    }

    data = {
        "model": "claude-3-5-haiku-20241022",
        "max_tokens": 1000,
        "temperature": 0,
        "system": "You are an AI assistant, you can help to extract claims from a model-generated response for a question. In addition, you should attribute the claims to the sentences followed by the sentence ids.",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    try:
        # pass SSL verification
        response = requests.post(
            url_anthropic,
            headers=headers,
            json=data,
            proxies=proxies,
            timeout=30,
        )

        if response.status_code == 200:
            result = response.json()
            return result['content'][0]['text'].split('\n')  # Return a line-separated list of claims.
        else:
            print(f"extract_triplets_to_claims Error: {response.status_code}")
            return []

    except requests.exceptions.RequestException as e:
        print("Request failed:", e)
        return []

## 2. Obtain Complete References

In [ ]:
def get_reference_for_claim(claim):
    prompt = f"""
    Please expand, provide additional relevant factual information and verify about the following claim:
    Claims: {claim}

    If the claim is accurate, not the hallucination and complete, return the original claim.
    If the claim is inaccurate, partial or lacking detail, return a corrected, a more detailed and comprehensive factual statement.
    """
    headers = {
        "x-api-key": api_key,
        "content-type": "application/json",
        "anthropic-version": "2023-06-01"
    }

    data = {
        "model": "claude-3-5-haiku-20241022",
        "max_tokens": 1000,
        "temperature": 0,
        "system": "You are an AI assistant who checks the factual accuracy of claims and returns corrected references if necessary.",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post(
            url_anthropic,
            headers=headers,
            json=data,
            proxies=proxies,
            timeout=30,
            verify=False
        )

        if response.status_code == 200:
            result = response.json()
            return result['content'][0]['text']
        else:
            print(f"Error verifying claim: {response.status_code}")
            return claim

    except requests.exceptions.RequestException as e:
        print("Verification request failed:", e)
        return claim

In [ ]:
def extract_and_get_references(claims, context):
    references = []
    for claim in claims:
        verified_reference = get_reference_for_claim(claim)
        references.append(verified_reference)

    final_reference = " ".join(references) + " " + context

    return final_reference

## 3. Validate claims, `model_input`, `model_output_text`, and References (Checker)

The validation results should be mapped back to the `model_output_text`, marking hallucination positions and probabilities, and outputting them as `soft_labels`.

In [ ]:
def extract_hallucination_positions(model_output_text, hallucination_results):
    # parse JSON data
    try:
        hallucination_results = json.loads(hallucination_results)
    except json.JSONDecodeError:
        print("Failed to decode JSON. Returning empty labels.")
        return {"soft_labels": []}

    soft_labels = []

    # find the position in the original text
    for result in hallucination_results:
        word = result['word']
        prob = result['prob']

        start = 0
        while True:
            start = model_output_text.find(word, start)
            if start == -1:
                break
            end = start + len(word)

            # save soft_labels
            soft_labels.append({
                "start": start,
                "end": end,
                "prob": prob
            })
            start = end

    return {"soft_labels": soft_labels}


In [ ]:
def triplets_and_references_checker(claims, model_output_text, references, question):
    prompt = f"""
    Evaluate the model output text for hallucinations by comparing it to the provided references, existed fact, claims, and question (model input). Identify any hallucinated or potentially inaccurate parts in the entire model output text. Highlight the hallucinated word and assign a probability of the hallucination word in the `model output text`.

    ### Question (Model Input)
    {question}

    ### Claims
    {claims}

    ### References
    {references}

    ### Model Output Text
    {model_output_text}

    ### Instructions
    1. Compare each claim with the provided references, question and existing fact (internal knowledge).
    2. If a claim cannot be fully supported by the references, identify the hallucinated words and mark it to `model output text`.
    3. Return character-level offsetss and assign hallucination probabilities.
    4. If the claim is fully supported, hallucination should not to be labeled.
    5. Assign hallucination probabilities based on the following criteria:
       - **0.7 - 1.0**: Fully fabricated or highly speculative content with no supporting evidence.
       - **0.4 - 0.7**: Partially incorrect or speculative content, but some evidence supports parts of the claim.
       - **0.1 - 0.4**: Minor inaccuracies, such as spelling errors, wrong formatting, or small factual deviations.
    6. Ensure that the hallucinated words do not overlap or repeat. If overlapping occurs, merge them or seperate them appropriately.
    7. Ensure the words are shown in the `model output text`.
    8. Highlight text in `model output text` that could potentially be a hallucination even if not explicitly listed in the claims.
    9. Return **all the hallucinated words or phrases** and assign each a hallucination probability (between 0 and 1).
    10. Do not filter out hallucinations based on low probability. Return results for any potential hallucination.
    11. Do not include any explanations, summaries, or additional text. **Return the JSON list directly.**
    12. Ensure all potential hallucinations are listed, even those with probabilities as low as 0.1.

    ### Output Example
    Only return results with all hallucinated words or phrases and their probability **strictly in the following JSON format**:
    [
        {{"word": <example_word>, "prob": <probability>}},
        {{"word": <another_word>, "prob": <probability>}}
    ]

    """

    headers = {
        "x-api-key": api_key,
        "content-type": "application/json",
        "anthropic-version": "2023-06-01"
    }

    data = {
        "model": "claude-3-5-haiku-20241022",
        "max_tokens": 1000,
        "temperature": 0,
        "system": "You are an AI assistant who checks the factual accuracy of claims and returns position and probability of the hallucination from model output text",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post(
            url_anthropic,
            headers=headers,
            json=data,
            proxies=proxies,
            timeout=30,
            verify=False
        )
        if response.status_code == 200:
            raw_labels = response.json()['content'][0]['text']
            return extract_hallucination_positions(model_output_text, raw_labels)
        else:
            print(f"Error during hallucination detection: {response.status_code}")
            return {"soft_labels": []}
    except requests.RequestException as e:
        print("Hallucination detection request failed:", e)
        return {"soft_labels": []}

## Main Logic

In [ ]:
def hallucination_detect(question, model_output_text, context):
    claims = extract_triplets_to_claims(question, model_output_text)
    references = extract_and_get_references(claims, context)
    hallucination_results = triplets_and_references_checker(claims, model_output_text, references, question)
    # extract soft_labels
    soft_labels = hallucination_results.get("soft_labels", [])

    # compute hard_labels
    hard_labels = recompute_hard_labels(soft_labels)

    return soft_labels, hard_labels

## Apply on My Dataset

In [ ]:
# process the dataset and save the results
def process_dataset(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    input_files = glob.glob(os.path.join(input_folder, "*.jsonl"))

    with tqdm(total=len(input_files), desc="Processing Files", unit="file") as file_progress:
        for file_path in input_files:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = [json.loads(line) for line in f]

            output_data = []

            with tqdm(total=len(data), desc=f"Processing {os.path.basename(file_path)}", unit="entry", leave=False) as entry_progress:
                for entry in data:
                    try:
                        question = entry.get("model_input", "")
                        model_output_text = entry.get("model_output_text", "")
                        context = entry.get("context_googlecse", "")

                        soft_labels, hard_labels = hallucination_detect(
                            question, model_output_text, context
                        )

                        output_entry = {
                            "id": entry.get("id"),
                            "lang": entry.get("lang"),
                            "model_input": entry.get("model_input"),
                            "model_output_text": entry.get("model_output_text"),
                            "model_id": entry.get("model_id"),
                            "soft_labels": soft_labels,
                            "hard_labels": hard_labels,
                            "model_output_logits": entry.get("model_output_logits"),
                            "model_output_tokens": entry.get("model_output_tokens")
                        }

                        output_data.append(output_entry)

                    except Exception as e:
                        print(f"Error processing entry {entry.get('id')}: {e}")

                    entry_progress.update(1)

            output_file = os.path.join(output_folder, os.path.basename(file_path))
            with open(output_file, 'w', encoding='utf-8') as f:
                for item in output_data:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')

            file_progress.update(1)
            print(f"Processed and saved: {output_file}")

In [ ]:
# jsonl files with context(Google API)
input_folder = "data/val/exknowledge/"
# output detected jsonl files
output_folder = "data/val/detect_2/"

process_dataset(input_folder, output_folder)

Processing mushroom.ar-val.v2.jsonl:   8%|▊         | 4/50 [01:28<17:18, 22.57s/entry]

Failed to decode JSON. Returning empty labels.



Processing mushroom.ar-val.v2.jsonl:  10%|█         | 5/50 [01:43<14:47, 19.71s/entry]

Failed to decode JSON. Returning empty labels.



Processing mushroom.ar-val.v2.jsonl:  14%|█▍        | 7/50 [03:09<22:24, 31.27s/entry]

...

Failed to decode JSON. Returning empty labels.



Processing mushroom.zh-val.v2.jsonl:  98%|█████████▊| 49/50 [1:39:57<01:39, 99.55s/entry] 

Failed to decode JSON. Returning empty labels.



Processing Files: 100%|██████████| 10/10 [5:52:56<00:00, 2117.65s/file]                  

Processed and saved: data/val/detect_2/mushroom.zh-val.v2.jsonl


## Evaluation

In [ ]:
import pandas as pd
import json
import os
from scorer import load_jsonl_file_to_records, score_iou, score_cor, main, recompute_hard_labels
import argparse as ap
import ast

In [ ]:
def evaluate_iou_and_cor(val_dir, detect_dir, output_file):
    """
    Evaluate IoU and Spearman correlation between the reference (val) and detected (detect) files.

    :param val_dir: Directory containing the ground truth files (e.g., data/val/val/)
    :param detect_dir: Directory containing the detected files (e.g., data/detect/)
    :param output_file: Path to save the evaluation results (optional)
    """
    # List all files in the validation directory
    val_files = os.listdir(val_dir)
    detect_files = os.listdir(detect_dir)

    # Ensure that we are comparing the same files (same lang)
    for val_file in val_files:
        # Skip non-JSONL files
        if not val_file.endswith('.jsonl'):
            continue

        # Check if the corresponding detect file exists
        detect_file_path = os.path.join(detect_dir, val_file)

        if not os.path.exists(detect_file_path):
            print(f"Warning: {detect_file_path} not found, skipping.")
            continue

        # Load ground truth (val) and detected (detect) data
        ref_dicts = load_jsonl_file_to_records(os.path.join(val_dir, val_file))
        pred_dicts = load_jsonl_file_to_records(detect_file_path)

        # Calculate IoU and Spearman correlation
        try:
            ious, cors = main(ref_dicts, pred_dicts)
        except IndexError as e:
            print(f"IndexError occurred for file: {val_file}, skipping this file. Error: {e}")
            continue

        # Print or save the results
        print(f"Results for {val_file}:")
        print(f"  Mean IoU: {ious.mean():.8f}")
        print(f"  Mean Spearman Correlation: {cors.mean():.8f}")

        # Optionally, save the results to a file
        if output_file:
            with open(output_file, 'a', encoding='utf-8') as f:
                f.write(f"Results for {val_file}:\n")
                f.write(f"  Mean IoU: {ious.mean():.8f}\n")
                f.write(f"  Mean Spearman Correlation: {cors.mean():.8f}\n\n")

val_dir = 'data/val/val/'
detect_dir = 'data/val/detect_2/'
output_file = 'evaluation_results3.txt'
evaluate_iou_and_cor(val_dir, detect_dir, output_file)

Results for mushroom.ar-val.v2.jsonl:
  Mean IoU: 0.38188029
  Mean Spearman Correlation: 0.32331063
Results for mushroom.de-val.v2.jsonl:
  Mean IoU: 0.41459801
  Mean Spearman Correlation: 0.39725764
Results for mushroom.en-val.v2.jsonl:
  Mean IoU: 0.36303494
  Mean Spearman Correlation: 0.40384660
Results for mushroom.es-val.v2.jsonl:
  Mean IoU: 0.36105925
  Mean Spearman Correlation: 0.36397433
Results for mushroom.fi-val.v2.jsonl:
  Mean IoU: 0.31845450
  Mean Spearman Correlation: 0.33442559
Results for mushroom.fr-val.v2.jsonl:
  Mean IoU: 0.30727328
  Mean Spearman Correlation: 0.35717161
Results for mushroom.hi-val.v2.jsonl:
  Mean IoU: 0.39672349
  Mean Spearman Correlation: 0.42207792
Results for mushroom.it-val.v2.jsonl:
  Mean IoU: 0.29420150
  Mean Spearman Correlation: 0.35644268
Results for mushroom.sv-val.v2.jsonl:
  Mean IoU: 0.51249971
  Mean Spearman Correlation: 0.42565442
Results for mushroom.zh-val.v2.jsonl:
  Mean IoU: 0.23721623
  Mean Spearman Correlation: 0